# Make a test STAC

## 1. Construct a static STAC

Guided by the tutorial at https://stacspec.org/en/tutorials/4-create-stac-collection/, make a STAC utilising the crevasse dataset of Chudley (2022, https://doi.org/10.5281/zenodo.6779087).

NB: Explore `rio-stac` for generating Items from rioxarray.

In [ ]:
import rioxarray as rxr
import pystac
import geopandas as gdf

from shapely.geometry import Polygon, mapping, box, shape
from datetime import datetime, timezone
from pystac.extensions.eo import Band, EOExtension
from pystac.extensions.raster import RasterBand, RasterExtension
from pystac.extensions.projection import ProjectionExtension


In [100]:

# v1.0 data is 6779088
# v1.1 data is 18631893

fpaths_2m = {
    'CW': 'https://zenodo.org/records/18631893/files/crevasse_binary_CW_2m.tif',
    'NE': 'https://zenodo.org/records/18631893/files/crevasse_binary_NE_2m.tif',
    'NO': 'https://zenodo.org/records/18631893/files/crevasse_binary_NO_2m.tif',
    'NW': 'https://zenodo.org/records/18631893/files/crevasse_binary_NW_2m.tif',
    'SE': 'https://zenodo.org/records/18631893/files/crevasse_binary_SE_2m.tif',
    'SE': 'https://zenodo.org/records/18631893/files/crevasse_binary_SW_2m.tif',
}

fpaths_200m = {
    'CW': 'https://zenodo.org/records/18631893/files/crevasse_fraction_CW_200m.tif',
    'NE': 'https://zenodo.org/records/18631893/files/crevasse_fraction_NE_200m.tif', 
    'NO': 'https://zenodo.org/records/18631893/files/crevasse_fraction_NO_200m.tif', 
    'NW': 'https://zenodo.org/records/18631893/files/crevasse_fraction_NW_200m.tif', 
    'SE': 'https://zenodo.org/records/18631893/files/crevasse_fraction_SE_200m.tif', 
    'SE': 'https://zenodo.org/records/18631893/files/crevasse_fraction_SW_200m.tif',
}

In [101]:
def get_bbox_and_footprint(fpath):
    with rxr.open_rasterio(fpath) as da:
        
        bounds = da.rio.bounds()
        bbox_3413 = [bounds[0], bounds[1], bounds[2], bounds[3]]
        footprint_3413 = box(bounds[0], bounds[1], bounds[2], bounds[3])
        transform_3413 = da.rio.transform()
        shape = da.rio.shape

        gdf_4326 = gdf.GeoDataFrame(geometry=[footprint_3413], crs='EPSG:3413').to_crs('EPSG:4326')
        bbox_4326 = gdf_4326.total_bounds.tolist()
        footprint_4326 = gdf_4326.geometry.values[0]

        return bbox_3413, mapping(footprint_3413), transform_3413, bbox_4326, mapping(footprint_4326), shape


In [102]:
def create_binary_stac_item_from_raster(raster_path, item_id):

    bbox_3413, footprint_3413, transform_3413, bbox_4326, footprint_4326, shape = get_bbox_and_footprint(raster_path)

    collection_item = pystac.Item(
        id = item_id,
        geometry=footprint_4326,
        bbox = bbox_4326,
        datetime = datetime(2018, 1, 1, tzinfo=timezone.utc),
        properties = {},
    )

    # Add item metadata
    collection_item.common_metadata.gsd = 2

    proj_ext = ProjectionExtension.ext(collection_item, add_if_missing=True)
    proj_ext.epsg = 3413
    proj_ext.code = 'EPSG:3413'
    proj_ext.bbox = bbox_3413
    proj_ext.geometry = footprint_3413
    proj_ext.transform = transform_3413
    proj_ext.shape = shape

    # Add asset and appropriate metadata
    asset = pystac.Asset(href = raster_path, media_type = pystac.MediaType.COG, )  # GEOTIFF / COG
    collection_item.add_asset(key = 'binary', asset = asset)

    raster = RasterExtension.ext(collection_item.assets['binary'], add_if_missing=True)
    raster.bands = [
        RasterBand.create(nodata=0, spatial_resolution=2, unit="m", data_type="uint8")
    ]

    # eo = EOExtension.ext(collection_item.assets["binary"], add_if_missing=True)
    # eo.apply(bands_binary)

    return collection_item, footprint_4326


def create_fraction_stac_item_from_raster(raster_path, item_id):

    bbox_3413, footprint_3413,transform_3413, bbox_4326, footprint_4326, shape = get_bbox_and_footprint(raster_path)

    collection_item = pystac.Item(
        id = item_id, 
        geometry=footprint_4326,
        bbox = bbox_4326, 
        datetime = datetime(2018, 1, 1, tzinfo=timezone.utc), 
        properties = {}, 
    ) 
    
    # Add item metadata
    collection_item.common_metadata.gsd = 200
    
    proj_ext = ProjectionExtension.ext(collection_item, add_if_missing=True)
    proj_ext.epsg = 3413
    proj_ext.code = 'EPSG:3413'
    proj_ext.bbox = bbox_3413
    proj_ext.geometry = footprint_3413
    proj_ext.transform = transform_3413
    proj_ext.shape = shape
    
    # Add asset and appropriate metadata
    asset = pystac.Asset(href = raster_path, media_type = pystac.MediaType.COG, )   # GEOTIFF / COG
    collection_item.add_asset(key = 'fraction', asset = asset) 
    raster = RasterExtension.ext(collection_item.assets['fraction'], add_if_missing=True)
    raster.bands = [
        RasterBand.create(nodata=-9999, spatial_resolution=200, unit="m", data_type="float32")
    ]

    
    # eo = EOExtension.ext(collection_item.assets["fraction"], add_if_missing=True) 
    # eo.apply(bands_fraction)

    return collection_item, footprint_4326



In [103]:
# loop through raster fpaths, collecting output items and footprints in a list

items_binary = []
footprints_binary = [] 

for key, fpath in fpaths_2m.items(): 
    item, footprint = create_binary_stac_item_from_raster(fpath, item_id = f'crevasse_binary_{key}_2m') 
    items_binary.append(item) 
    footprints_binary.append(footprint) 

items_fraction = [] 
footprints_fraction = [] 

for key, fpath in fpaths_200m.items(): 
    item, footprint = create_fraction_stac_item_from_raster(fpath, item_id = f'crevasse_fraction_{key}_200m') 
    items_fraction.append(item) 
    footprints_fraction.append(footprint)


In [104]:
# get union of footprints for each resolution, to use as collection geometry
union_footprint_binary = shape(footprints_binary[0]) 
for footprint in footprints_binary[1:]: 
    union_footprint_binary = union_footprint_binary.union(shape(footprint)) 

union_footprint_fraction = shape(footprints_fraction[0]) 
for footprint in footprints_fraction[1:]: 
    union_footprint_fraction = union_footprint_fraction.union(shape(footprint))

# get bbox of union footprints
bbox_binary = union_footprint_binary.bounds 
bbox_fraction = union_footprint_fraction.bounds

# Get spatial extents
spatial_extent_binary = pystac.SpatialExtent(bboxes=[bbox_binary])
spatial_extent_fraction = pystac.SpatialExtent(bboxes=[bbox_fraction])

# Set collection interval to beginning of 2018, as this is when the data was collected
temporal_extent = pystac.TemporalExtent(intervals=[(datetime(2018, 1, 1, tzinfo=timezone.utc), None)])

# create extents
extent_binary = pystac.Extent(spatial=spatial_extent_binary, temporal=temporal_extent) 
extent_fraction = pystac.Extent(spatial=spatial_extent_fraction, temporal=temporal_extent)


In [105]:
# Construct collection
collection_fraction = pystac.Collection(
    id = 'crevasse_fraction',
    description = 'Fractional classification of crevasse presence (0-1) across the Greenland Ice Sheet from ArcticDEM v3.0 data at 200m resolution', 
    extent = extent_fraction, 
    license = 'CC-BY-4.0', 
    )

collection_fraction.add_items(items_fraction)


collection_binary = pystac.Collection(
    id = 'crevasse_binary',
    description = 'Binary classification of crevasse presence (2) or absence (1) across the Greenland Ice Sheet from ArcticDEM v3.0 data at 2m resolution', 
    extent = extent_binary, 
    license = 'CC-BY-4.0', 
    )

collection_binary.add_items(items_binary)

[<Link rel=item target=<Item id=crevasse_binary_CW_2m>>,
 <Link rel=item target=<Item id=crevasse_binary_NE_2m>>,
 <Link rel=item target=<Item id=crevasse_binary_NO_2m>>,
 <Link rel=item target=<Item id=crevasse_binary_NW_2m>>,
 <Link rel=item target=<Item id=crevasse_binary_SE_2m>>]

In [106]:
items_fraction[0]

<Item id=crevasse_fraction_CW_200m>

In [107]:
catalog = pystac.Catalog(
    id = 'greenland-crevasses-arcticdem',
    description='STAC collection of binary and fractional classifications of crevasse presence across the Greenland Ice Sheet, derived from ArcticDEM v3.0 data at 2m and 200m resolution',
) 

catalog.add_child(collection_binary) 
catalog.add_child(collection_fraction)

<Link rel=child target=<Collection id=crevasse_fraction>>

In [108]:
catalog.describe()

* <Catalog id=greenland-crevasses-arcticdem>
    * <Collection id=crevasse_binary>
      * <Item id=crevasse_binary_CW_2m>
      * <Item id=crevasse_binary_NE_2m>
      * <Item id=crevasse_binary_NO_2m>
      * <Item id=crevasse_binary_NW_2m>
      * <Item id=crevasse_binary_SE_2m>
    * <Collection id=crevasse_fraction>
      * <Item id=crevasse_fraction_CW_200m>
      * <Item id=crevasse_fraction_NE_200m>
      * <Item id=crevasse_fraction_NO_200m>
      * <Item id=crevasse_fraction_NW_200m>
      * <Item id=crevasse_fraction_SE_200m>


In [ ]:
catalog.normalize_and_save(
    root_href = './stac-api/data/stac-catalog/',
    catalog_type=pystac.CatalogType.SELF_CONTAINED,
)

## 2. Test dynamic STAC API

Must have the API running to do this - see `README.md`.

In [11]:
import pystac_client
import stackstac
import geopandas as gpd

from shapely.geometry import box

In [7]:
catalog = pystac_client.Client.open('http://localhost:8080/')

In [8]:
catalog

<Client id=stac-fastapi>

In [9]:
[c for c in catalog.get_collections()]

[<CollectionClient id=crevasse_binary>,
 <CollectionClient id=crevasse_fraction>]

In [12]:

# test bounds that cut across two tiles, for stackstac purposes
bounds = -269130, -1999768, -161007, -1909081, 

gdf_3413 = gpd.GeoDataFrame(geometry=[box(*bounds)], crs="epsg:3413")
gdf_4326 = gdf_3413.to_crs(4326)

bounds_4326 = gdf_4326.geometry.values[0].bounds

# Get EPSG:4326 geometry for feeding in to STAC search
geom_4326 = gdf_4326.geometry.values[0]

In [19]:
collection = "crevasse_fraction"
# collection = "crevasse_binary"
geom = geom_4326

search = catalog.search(
    collections=[collection],
    intersects=geom,
)

# Check how many items were returned
items = search.item_collection()
print(f"Query returns {len(items)} Items")

Query returns 2 Items


We can use `stackstac` to merge our query.

In [20]:
data = stackstac.stack(
    items,
    bounds=bounds,
    # chunksize=512,
    epsg=3413,
)

# Compress along 'time' dimension to get single msoaic of all tiles
data = stackstac.mosaic(data, dim='time').squeeze()

In [21]:
data

<xarray.DataArray 'stackstac-933541fb647d221e350f37ec497ae05b' (y: 455, x: 425)> Size: 2MB
dask.array<getitem, shape=(455, 425), dtype=float64, chunksize=(455, 425), chunktype=numpy.ndarray>
Coordinates:
  * y             (y) float64 4kB -1.909e+06 -1.909e+06 ... -1.999e+06 -2e+06
  * x             (x) float64 3kB -2.693e+05 -2.69e+05 ... -1.614e+05 -1.612e+05
    band          <U8 32B 'fraction'
    proj:code     <U9 36B 'EPSG:3413'
    gsd           int64 8B 200
    raster:bands  object 8B {'unit': 'm', 'nodata': -9999, 'data_type': 'floa...
    epsg          int64 8B 3413
Attributes:
    spec:           RasterSpec(epsg=3413, bounds=(-269290.0613139182, -199989...
    crs:            epsg:3413
    transform:      | 255.01, 0.00,-269290.06|\n| 0.00,-199.65,-1909056.28|\n...
    resolution_xy:  (255.00952775939223, 199.65031197637146)

NB: This may occasionally plot, but Zenodo is not designed to allow for this kind of cloud-optimised, parallel access. Trying to download the full dataset will mostly just return an error. However, the proof-of-concept works: we just need a proper way of hosting cloud data.